In [1]:
!apt update
!apt install xvfb
!pip install gym-notebook-wrapper
!pip install pyvirtualdisplay
!pip install gym==0.9.7
!pip uninstall matplotlib -y
!pip install matplotlib==2.2.5
!pip install JSAnimation
!pip uninstall pyglet -y
!pip install pyglet==1.2.4

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downlo

  Created wheel for JSAnimation: filename=JSAnimation-0.1-cp37-none-any.whl size=11426 sha256=5bd2345adc4a7e64ea4dcca2cdf9536fc722fef87fd82f7096be3b446721d31f
  Stored in directory: /root/.cache/pip/wheels/3c/c2/b2/b444dffc3eed9c78139288d301c4009a42c0dd061d3b62cead
Successfully built JSAnimation
Uninstalling pyglet-1.5.0:
  Successfully uninstalled pyglet-1.5.0
     |████████████████████████████████| 972kB 5.6MB/s 
ERROR: dopamine-rl 1.0.5 has requirement gym>=0.10.5, but you'll have gym 0.9.7 which is incompatible.


In [2]:
# 使用するパッケージの宣言
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym
from pyvirtualdisplay import Display
import gnwrapper
import gym
from collections import namedtuple
import random
import torch
from torch import optim
from torch import nn
import torch.nn.functional as F

d = Display()
d.start()

In [3]:
Transition = namedtuple(
    'Transition', ('state', 'action', 'next_state', 'reward')
)

ENV = 'CartPole-v1'
GAMMA = 0.99
MAX_STEPS = 200
NUM_EPISODES = 1000

BATCH_SIZE = 64
CAPACITY = 10000

In [4]:
# 経験を保存するメモリクラスを定義します

class ReplayMemory:

    def __init__(self, CAPACITY):
        self.capacity = CAPACITY  # メモリの最大長さ
        self.memory = []  # 経験を保存する変数
        self.index = 0  # 保存するindexを示す変数

    def push(self, state, action, state_next, reward):
        '''transition = (state, action, state_next, reward)をメモリに保存する'''

        if len(self.memory) < self.capacity:
            self.memory.append(None)  # メモリが満タンでないときは足す

        # namedtupleのTransitionを使用し、値とフィールド名をペアにして保存します
        self.memory[self.index] = Transition(state, action, state_next, reward)

        self.index = (self.index + 1) % self.capacity  # 保存するindexを1つずらす

    def sample(self, batch_size):
        '''batch_size分だけ、ランダムに保存内容を取り出す'''
        return random.sample(self.memory, batch_size)

    def __len__(self):
        '''関数lenに対して、現在の変数memoryの長さを返す'''
        return len(self.memory)

In [5]:
class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions  # CartPoleの行動（右に左に押す）の2を取得

        # 経験を記憶するメモリオブジェクトを生成
        self.memory = ReplayMemory(CAPACITY)

        # ニューラルネットワークを構築
        self.model = nn.Sequential()
        self.model.add_module('fc1', nn.Linear(num_states, 32))
        self.model.add_module('relu1', nn.ReLU())
        self.model.add_module('fc2', nn.Linear(32, 32))
        self.model.add_module('relu2', nn.ReLU())
        self.model.add_module('fc3', nn.Linear(32, num_actions))

        print(self.model)  # ネットワークの形を出力

        # 最適化手法の設定
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)

    def replay(self):
        '''Experience Replayでネットワークの結合パラメータを学習'''

        # -----------------------------------------
        # 1. メモリサイズの確認
        # -----------------------------------------
        # 1.1 メモリサイズがミニバッチより小さい間は何もしない
        if len(self.memory) < BATCH_SIZE:
            return

        # -----------------------------------------
        # 2. ミニバッチの作成
        # -----------------------------------------
        # 2.1 メモリからミニバッチ分のデータを取り出す
        transitions = self.memory.sample(BATCH_SIZE)

        # 2.2 各変数をミニバッチに対応する形に変形
        # transitionsは1stepごとの(state, action, state_next, reward)が、BATCH_SIZE分格納されている
        # つまり、(state, action, state_next, reward)×BATCH_SIZE
        # これをミニバッチにしたい。つまり
        # (state×BATCH_SIZE, action×BATCH_SIZE, state_next×BATCH_SIZE, reward×BATCH_SIZE)にする
        batch = Transition(*zip(*transitions))


        # 2.3 各変数の要素をミニバッチに対応する形に変形し、ネットワークで扱えるようVariableにする
        # 例えばstateの場合、[torch.FloatTensor of size 1x4]がBATCH_SIZE分並んでいるのですが、
        # それを torch.FloatTensor of size BATCH_SIZEx4 に変換します
        # 状態、行動、報酬、non_finalの状態のミニバッチのVariableを作成
        # catはConcatenates（結合）のことです。
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        non_final_next_states = torch.cat([s for s in batch.next_state
                                           if s is not None])

        # -----------------------------------------
        # 3. 教師信号となるQ(s_t, a_t)値を求める
        # -----------------------------------------
        # 3.1 ネットワークを推論モードに切り替える
        self.model.eval()


        # 3.2 ネットワークが出力したQ(s_t, a_t)を求める
        # self.model(state_batch)は、右左の両方のQ値を出力しており
        # [torch.FloatTensor of size BATCH_SIZEx2]になっている。
        # ここから実行したアクションa_tに対応するQ値を求めるため、action_batchで行った行動a_tが右か左かのindexを求め
        # それに対応するQ値をgatherでひっぱり出す。
        state_action_values = self.model(state_batch).gather(1, action_batch)
        # 3.3 max{Q(s_t+1, a)}値を求める。ただし次の状態があるかに注意。

        # cartpoleがdoneになっておらず、next_stateがあるかをチェックするインデックスマスクを作成
        non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None,
                                                    batch.next_state)))
        
        # まずは全部0にしておく
        next_state_values = torch.zeros(BATCH_SIZE)

        # 次の状態があるindexの最大Q値を求める
        # 出力にアクセスし、max(1)で列方向の最大値の[値、index]を求めます
        # そしてそのQ値（index=0）を出力します
        # detachでその値を取り出します
        next_state_values[non_final_mask] = self.model(non_final_next_states).max(1)[0].detach()

        # 3.4 教師となるQ(s_t, a_t)値を、Q学習の式から求める
        expected_state_action_values = reward_batch + GAMMA * next_state_values

        # -----------------------------------------
        # 4. 結合パラメータの更新
        # -----------------------------------------
        # 4.1 ネットワークを訓練モードに切り替える
        self.model.train()

        # 4.2 損失関数を計算する（smooth_l1_lossはHuberloss）
        # expected_state_action_valuesは
        # sizeが[minbatch]になっているので、unsqueezeで[minibatch x 1]へ
        loss = F.smooth_l1_loss(state_action_values,
                                expected_state_action_values.unsqueeze(1))

        # 4.3 結合パラメータを更新する
        self.optimizer.zero_grad()  # 勾配をリセット
        loss.backward()  # バックプロパゲーションを計算
        self.optimizer.step()  # 結合パラメータを更新

    def decide_action(self, state, episode):
        '''現在の状態に応じて、行動を決定する'''
        # ε-greedy法で徐々に最適行動のみを採用する
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            self.model.eval()  # ネットワークを推論モードに切り替える
            with torch.no_grad():
                action = self.model(state).max(1)[1].view(1, 1)
            # ネットワークの出力の最大値のindexを取り出します = max(1)[1]
            # .view(1,1)は[torch.LongTensor of size 1]　を size 1x1 に変換します

        else:
            # 0,1の行動をランダムに返す
            action = torch.LongTensor(
                [[random.randrange(self.num_actions)]])  # 0,1の行動をランダムに返す
            # actionは[torch.LongTensor of size 1x1]の形になります

        return action

In [6]:
# CartPoleで動くエージェントクラスです、棒付き台車そのものになります


class Agent:
    def __init__(self, num_states, num_actions):
        '''課題の状態と行動の数を設定する'''
        self.brain = Brain(num_states, num_actions)  # エージェントが行動を決定するための頭脳を生成

    def update_q_function(self):
        '''Q関数を更新する'''
        self.brain.replay()

    def get_action(self, state, episode):
        '''行動を決定する'''
        action = self.brain.decide_action(state, episode)
        return action

    def memorize(self, state, action, state_next, reward):
        '''memoryオブジェクトに、state, action, state_next, rewardの内容を保存する'''
        self.brain.memory.push(state, action, state_next, reward)

In [7]:
# CartPoleを実行する環境のクラスです


class Environment:

    def __init__(self):
        # self.env = gym.make(ENV)  # 実行する課題を設定
        self.env = gnwrapper.Monitor(gym.make('CartPole-v1'),directory="./",force=True) 
        num_states = self.env.observation_space.shape[0]  # 課題の状態数4を取得
        num_actions = self.env.action_space.n  # CartPoleの行動（右に左に押す）の2を取得
        self.agent = Agent(num_states, num_actions)  # 環境内で行動するAgentを生成

        
    def run(self):
        '''実行'''
        episode_10_list = np.zeros(10)  # 10試行分の立ち続けたstep数を格納し、平均ステップ数を出力に利用
        complete_episodes = 0  # 195step以上連続で立ち続けた試行数
        episode_final = False  # 最後の試行フラグ
        # frames = []  # 最後の試行を動画にするために画像を格納する変数

        for episode in range(NUM_EPISODES):  # 最大試行数分繰り返す
            observation = self.env.reset()  # 環境の初期化

            state = observation  # 観測をそのまま状態sとして使用
            state = torch.from_numpy(state).type(
                torch.FloatTensor)  # NumPy変数をPyTorchのテンソルに変換
            state = torch.unsqueeze(state, 0)  # size 4をsize 1x4に変換

            for step in range(MAX_STEPS):  # 1エピソードのループ

                # if episode_final is True:  # 最終試行ではframesに各時刻の画像を追加していく
                #     frames.append(self.env.render(mode='rgb_array'))

                action = self.agent.get_action(state, episode)  # 行動を求める

                # 行動a_tの実行により、s_{t+1}とdoneフラグを求める
                # actionから.item()を指定して、中身を取り出す
                observation_next, _, done, _ = self.env.step(
                    action.item())  # rewardとinfoは使わないので_にする

                # 報酬を与える。さらにepisodeの終了評価と、state_nextを設定する
                if done:  # ステップ数が200経過するか、一定角度以上傾くとdoneはtrueになる
                    state_next = None  # 次の状態はないので、Noneを格納

                    # 直近10episodeの立てたstep数リストに追加
                    episode_10_list = np.hstack(
                        (episode_10_list[1:], step + 1))

                    if step < 195:
                        reward = torch.FloatTensor(
                            [-1.0])  # 途中でこけたら罰則として報酬-1を与える
                        complete_episodes = 0  # 連続成功記録をリセット
                    else:
                        reward = torch.FloatTensor([1.0])  # 立ったまま終了時は報酬1を与える
                        complete_episodes = complete_episodes + 1  # 連続記録を更新
                else:
                    reward = torch.FloatTensor([0.0])  # 普段は報酬0
                    state_next = observation_next  # 観測をそのまま状態とする
                    state_next = torch.from_numpy(state_next).type(
                        torch.FloatTensor)  # numpy変数をPyTorchのテンソルに変換
                    state_next = torch.unsqueeze(state_next, 0)  # size 4をsize 1x4に変換

                # メモリに経験を追加
                self.agent.memorize(state, action, state_next, reward)

                # Experience ReplayでQ関数を更新する
                self.agent.update_q_function()

                # 観測の更新
                state = state_next

                # 終了時の処理
                if done:
                    print('%d Episode: Finished after %d steps：10試行の平均step数 = %.1lf' % (
                        episode, step + 1, episode_10_list.mean()))
                    break

            if episode_final is True:
                # 動画を保存と描画
                # display_frames_as_gif(frames)
                self.env.display()
                break

            # 10連続で200step経ち続けたら成功
            if complete_episodes >= 10:
                print('10回連続成功')
                episode_final = True  # 次の試行を描画を行う最終試行とする

In [8]:
# main クラス
cartpole_env = Environment()
cartpole_env.run()

/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Sequential(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)
0 Episode: Finished after 11 steps：10試行の平均step数 = 1.1
1 Episode: Finished after 9 steps：10試行の平均step数 = 2.0
2 Episode: Finished after 11 steps：10試行の平均step数 = 3.1
3 Episode: Finished after 13 steps：10試行の平均step数 = 4.4
4 Episode: Finished after 9 steps：10試行の平均step数 = 5.3
5 Episode: Finished after 8 steps：10試行の平均step数 = 6.1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)


6 Episode: Finished after 11 steps：10試行の平均step数 = 7.2
7 Episode: Finished after 8 steps：10試行の平均step数 = 8.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)


...

343 Episode: Finished after 196 steps：10試行の平均step数 = 187.1
355 Episode: Finished after 113 steps：10試行の平均step数 = 181.2
357 Episode: Finished after 8 steps：10試行の平均step数 = 162.5
358 Episode: Finished after 9 steps：10試行の平均step数 = 145.1
359 Episode: Finished after 103 steps：10試行の平均step数 = 135.4
360 Episode: Finished after 10 steps：10試行の平均step数 = 118.1
361 Episode: Finished after 17 steps：10試行の平均step数 = 101.5
362 Episode: Finished after 10 steps：10試行の平均step数 = 84.2
363 Episode: Finished after 9 steps：10試行の平均step数 = 66.7
364 Episode: Finished after 9 steps：10試行の平均step数 = 48.4
365 Episode: Finished after 10 steps：10試行の平均step数 = 29.8
385 Episode: Finished after 177 steps：10試行の平均step数 = 36.2
390 Episode: Finished after 185 steps：10試行の平均step数 = 53.9
398 Episode: Finished after 187 steps：10試行の平均step数 = 71.7
510 Episode: Finished after 147 steps：10試行の平均step数 = 76.1
511 Episode: Finished after 93 steps：10試行の平均step数 = 84.4


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)


512 Episode: Finished after 23 steps：10試行の平均step数 = 85.0
620 Episode: Finished after 146 steps：10試行の平均step数 = 98.6
624 Episode: Finished after 71 steps：10試行の平均step数 = 104.8
628 Episode: Finished after 185 steps：10試行の平均step数 = 122.4
629 Episode: Finished after 123 steps：10試行の平均step数 = 133.7
630 Episode: Finished after 122 steps：10試行の平均step数 = 128.2
686 Episode: Finished after 154 steps：10試行の平均step数 = 125.1
701 Episode: Finished after 106 steps：10試行の平均step数 = 117.0
702 Episode: Finished after 9 steps：10試行の平均step数 = 103.2
703 Episode: Finished after 9 steps：10試行の平均step数 = 94.8
704 Episode: Finished after 10 steps：10試行の平均step数 = 93.5
705 Episode: Finished after 10 steps：10試行の平均step数 = 79.9
706 Episode: Finished after 10 steps：10試行の平均step数 = 73.8
707 Episode: Finished after 10 steps：10試行の平均step数 = 56.3
708 Episode: Finished after 10 steps：10試行の平均step数 = 45.0
709 Episode: Finished after 9 steps：10試行の平均step数 = 33.7
710 Episode: Finished after 9 steps：10試行の平均step数 = 19.2
711 Episode: Finished 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)


729 Episode: Finished after 9 steps：10試行の平均step数 = 9.2
730 Episode: Finished after 10 steps：10試行の平均step数 = 9.1
731 Episode: Finished after 9 steps：10試行の平均step数 = 9.1
732 Episode: Finished after 9 steps：10試行の平均step数 = 9.2
733 Episode: Finished after 10 steps：10試行の平均step数 = 9.3
734 Episode: Finished after 9 steps：10試行の平均step数 = 9.3
735 Episode: Finished after 9 steps：10試行の平均step数 = 9.4
736 Episode: Finished after 9 steps：10試行の平均step数 = 9.3
737 Episode: Finished after 10 steps：10試行の平均step数 = 9.4
738 Episode: Finished after 10 steps：10試行の平均step数 = 9.4
739 Episode: Finished after 9 steps：10試行の平均step数 = 9.4
740 Episode: Finished after 10 steps：10試行の平均step数 = 9.4
741 Episode: Finished after 9 steps：10試行の平均step数 = 9.4
742 Episode: Finished after 10 steps：10試行の平均step数 = 9.5
743 Episode: Finished after 9 steps：10試行の平均step数 = 9.4
744 Episode: Finished after 10 steps：10試行の平均step数 = 9.5
745 Episode: Finished after 8 steps：10試行の平均step数 = 9.4
746 Episode: Finished after 9 steps：10試行の平均step数 = 9.4
747